# Tool explicitly called

In [1]:
from datetime import datetime
from langchain_core.tools import StructuredTool
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import os


In [2]:
# Asegúrate de tener tu clave en la variable de entorno OPENAI_KEY
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

# 1. Definir la función para obtener la fecha actual
def get_current_date_tool():
    """Devuelve la fecha y hora actuales."""
    now = datetime.now()
    return now.strftime("%A, %d de %B de %Y")

# 2. Crear la herramienta estructurada a partir de la función
date_tool = StructuredTool.from_function(
    func=get_current_date_tool,
    name="get_current_date",
    description="Devuelve la fecha y hora actuales."
)

# 3. Inicializar el modelo de lenguaje
model = ChatOpenAI(temperature=0.7)

# 4. Crear un MemorySaver para el historial de la conversación
memory = MemorySaver()

# 5. Definir la plantilla de prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente conversacional amigable y útil."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [3]:
# 6. Definir el estado inicial
def get_state():
    return {"history": []}

# 7. Definir el paso de conversación
def conversation_step(state, input):
    # Obtener el historial de la conversación
    history = state["history"]
    
    # Formatear el input con el historial para el modelo
    messages = prompt.format_messages(history=history, input=input)
    
    # Obtener la respuesta del modelo
    response = model.invoke(messages)
    
    # Verificar si la respuesta contiene una solicitud para obtener la fecha actual
    if "¿Qué día es hoy?" in input or "¿Cuál es la fecha de hoy?" in input:
        # Usar la herramienta para obtener la fecha actual
        current_date = date_tool.invoke({})
        response = AIMessage(content=f"La fecha de hoy es {current_date}.")
    
    # Actualizar el historial con los nuevos mensajes
    new_history = history + [HumanMessage(content=input), response]
    
    # Devolver el estado actualizado
    return {"history": new_history, "response": response.content}

In [4]:
# Initialize state
state = get_state()

# Process first user input
result = conversation_step(state, "Hola, Mi nombre es Kike")
state = {"history": result["history"]}  # Update state
print(result)

# Process follow-up question
result = conversation_step(state, "¿Qué día es hoy?")
# result = conversation_step(state, "¿Quien descibrio el continente americano?")
state = {"history": result["history"]}  # Update state
print(result["response"])

# Process follow-up question
result = conversation_step(state, "¿Recuerdas como me llamo?")
state = {"history": result["history"]}  # Update state
print(result["response"])

{'history': [HumanMessage(content='Hola, Mi nombre es Kike', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola, Kike! ¡Es un placer conocerte! ¿En qué puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 33, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BdRMNrNmA7ST0BdKRBix1sFUqxP64', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3c161609-d74b-4459-9232-dabc45bc0154-0', usage_metadata={'input_tokens': 33, 'output_tokens': 21, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})], 'response': '¡Hola, Kike! ¡Es un placer co